In [15]:
import requests
import requests_cache
import lxml
from bs4 import BeautifulSoup as bs
import json
import urllib2

requests_cache.install_cache("cache")

In [22]:
temp = json.load(urllib2.urlopen("http://api.wunderground.com/api/7c7b33990456667c/geolookup//history_20080105/q/zmw:00000.1.54511.json"))
temp

{u'history': {u'dailysummary': [{u'coolingdegreedays': u'0',
    u'coolingdegreedaysnormal': u'',
    u'date': {u'hour': u'00',
     u'mday': u'05',
     u'min': u'00',
     u'mon': u'01',
     u'pretty': u'12:00 AM CST on January 05, 2008',
     u'tzname': u'Asia/Shanghai',
     u'year': u'2008'},
    u'fog': u'0',
    u'gdegreedays': u'0',
    u'hail': u'0',
    u'heatingdegreedays': u'38',
    u'heatingdegreedaysnormal': u'',
    u'humidity': u'54',
    u'maxdewpti': u'17',
    u'maxdewptm': u'-9',
    u'maxhumidity': u'79',
    u'maxpressurei': u'30.21',
    u'maxpressurem': u'1023',
    u'maxtempi': u'40',
    u'maxtempm': u'4',
    u'maxvisi': u'7.0',
    u'maxvism': u'11.0',
    u'maxwspdi': u'7',
    u'maxwspdm': u'11',
    u'meandewpti': u'11',
    u'meandewptm': u'-11',
    u'meanpressurei': u'30.12',
    u'meanpressurem': u'1019.84',
    u'meantempi': u'28',
    u'meantempm': u'-2',
    u'meanvisi': u'2.8',
    u'meanvism': u'4.6',
    u'meanwdird': u'352',
    u'meanwdire':